    * 장영재 교수님의 2021강의를 정리한 것입니다.



### [ 학습목표 ]

- 나무모형을 이해한다.
- 나무모형 분할 법칙을 익힌다.
- 나무모형의 가지치기 원리를 이해한다.
- 적절한 크기의 나무모형을 선택하는 방법을 이해한다.

### [ 주요용어 ]

- `가지치기`: 나무모형을 최대로 만든 후, 가지가 많은 (노드 수가 많은) 나무로부터 분류성능 향상에 큰 도움이 되지 못하는 가지를 잘라내는 것, 교차타당성(cross-validation)방법을 통해 최적크기의 나무모형을 선택하는 방법
- `분할`: 나무구조의 매 단계마다 가급적 같은 성질의 관찰치들이 모이도록 집단을 나누어가는 것
- `불순도`: 이질적인 자료가 포함되어 있는 정도
- `분류나무`: 목표변수가 `범주형 변수`일때의 나무모형
- `회귀나무모형`: 목표변수가 `연속형` 일때의 나무모형으로 나무구조의 회귀분석 모형

### [ 목 차 ]

1. 나무모형이란
2. 분류나무모형의 분할방법 - CART, C4.5, CHAID, CRUISE, GUIDE, QUEST
3. 분류나무모형의 크기 선택 - 분할정지 방법, 가지치기 방법 
4. 분류나무모형 사례분석 
5. 회귀나무모형의 분할방법 - CART, GUIDE
6. 회귀나무모형의 사례분석

# 1. 나무모형이란

---

## (1) 나무모형 소개

    - 나무모형은 분석과정을 나무구조로 도형화하여 분류분석 혹은 회귀분석을 수행    
    - 나무모형은 자료를 분할하여 여러 개의 하부 노드로 할당시키는 방법을 통해 예측모형을 구성      
    - 목표변수가 범주형이면 분류나무, 수치형이면 회귀나무 모형을 이용
    - 나무모형에 의한 의사결정 규칙은 데이터를 바탕으로 이루어짐
        : 수집된 데이터를 이용하여 분리규칙을 이용해 뿌리노드, 중간노드, 최종노드를 통하여 의사결정 수행

## (2) 나무모형 목적

(ex) 카드발급 승인에 대한 나무모형
![02_01_Tree_Model.png](attachment:02_01_Tree_Model.png)

- **분류**: 카드발급 승인 or 거절에 대한 의사결정을 내려야 하는 경우, 의사결정에 이르는 규칙도 개발 필요

- **예측**: 카드 신청자들이 발급 후에 어떤 신용상태가 될지 예측, 월간 평균 신용카드 사용액 예측

- **등급화**: 신용카드 발급자들을 일정 기준으로 등급을 나누어 신용한도액 차등 설정

- **세분화**: 군집분석으로 고객들을 군집으로 나누어 그 결과를 목표변수로 사용하여  군집별 특징 찾기, 이를 활용하여 신규 고객에 대한 고객 세분화도 가능

- **변수선택**: 데이터 안의 변수들 중 중요한 것과 제외해도 문제가 없을 변수 선택

- **상호작용 탐색**: 입력변수 중 일부 변수의 특별한 조합이 가지는 특별한 효과를 찾아내는 것, 추후분석에서는 나무모형이 찾아낸 상호작용을 본 모형에 포함하여 적합


## (3) 상호작용의 발견

- 입력변수의 개수가 많은 경우 고려해야 하는 상호작용의 개수가 매우 많아져서 많은 시간과 노력이 소요됨
- (ex) 타이타닉 승선자의 생존률 분석: 좌석등급(1등석>2등석>3등석>승무원), 성별(여성>남성), 연령(어린이>성인)은 생존과 사망을 구분 짓는 매우 중요한 역할을 하는 변수
- 상호작용이 없는 로지스틱 회귀분석이을 이용하여 문석해 보면 유의한 변수 확인 가능

In [1]:
from sklearn.tree import DecisionTreeClassifier
import os 
import pandas as pd

# os 라이브러리:https://www.daleseo.com/python-os-environ/
    
titanic_path = os.getenv("HOME")+'/portfolio_mj/Data_Mining/data/titanic_train.csv'
titanic_data = pd.read_csv(titanic_path)

titanic_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
titanic_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [3]:
titanic_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
# 범주형 변수 => 수치형 변수
titanic_data['Sex'] = titanic_data['Sex'].map({'male':0, 'female':1})

titanic_data[['Sex']].head()

,Sex
0,0
1,1
2,1
3,1
4,0


In [13]:
# 결측치 처리
titanic_data['Age'].fillna(0)

0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
5       0.0
6      54.0
7       2.0
8      27.0
9      14.0
10      4.0
11     58.0
12     20.0
13     39.0
14     14.0
15     55.0
16      2.0
17      0.0
18     31.0
19      0.0
20     35.0
21     34.0
22     15.0
23     28.0
24      8.0
25     38.0
26      0.0
27     19.0
28      0.0
29      0.0
       ... 
861    21.0
862    48.0
863     0.0
864    24.0
865    42.0
866    27.0
867    31.0
868     0.0
869     4.0
870    26.0
871    47.0
872    33.0
873    47.0
874    28.0
875    15.0
876    20.0
877    19.0
878     0.0
879    56.0
880    25.0
881    33.0
882    22.0
883    28.0
884    25.0
885    39.0
886    27.0
887    19.0
888     0.0
889    26.0
890    32.0
Name: Age, Length: 891, dtype: float64

In [15]:
titanic_data['Age'].isnull().sum()

177

In [17]:
titanic_data['Age'].head()

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
Name: Age, dtype: float64

In [16]:
# 독립변수 종속변수 설정
X = titanic_data[['Pclass','Sex','Age']]
y = titanic_data[['Survived']]

# DecisionTree Classifier 생성
tree_titanic = DecisionTreeClassifier(max_depth=3)

# 학습
tree_titanic.fit(X, y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
# 트리 불러오기
import graphviz
from sklearn.tree import export_graphviz

# https://codedragon.tistory.com/10477
# export_graphviz(): 호출 결과로 out_file로 지정된 tree.dot 파일을 생성
export_graphviz(tree_titanic,\
               out_file = image_path("titanic_tree.dot"),\    # 파일저장 유무
               class_names = titanic_data.survive_names, \    # class변수 값 이름 설정
               feature_names = titanic_data.feature_names, \  # 컬럼 이름들 지정
               impurity=True, \
               filled=True)            # 노드의 클래스가 구분되도록 색칠됨

In [ ]:
with open("titanic_tree.dot") as f:
  dot_graph = f.read()
graphviz.Source(dot_graph)

- 장점: 분류분석과 회귀분석의 과정이 나무구조에 의해 표현되므로 분석자가 그 과정을 쉽게 이해하고 설명할 수 있음, 분석결과를 이해하기 쉽고 대용량 데이터에 대한 분류 및 에측이 용이하여 많은 분야에서 사용, 입력변수의 형태에 관계없이 적용할 수 있음, 결측치의 처리가 용이
- 단점: 분류성과 및 예측성과가 다른 모형보다 떨어질 수 있음, 데이터의 변형에 따라 나무모형이 쉽게 변경될 수 있음